In [ ]:
# ========== RADIAL TOKENIZER ==========
import torch
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from radial_tokenizer import RadialTokenizer  # Ensure same folder or adjust path

# ========== Configuration ==========
IMG_PATH = "C:/Users/denni/Downloads/test.png"  # Update if needed
IMG_NAME = "sample1"
OUTPUT_DIR = "../output"

# ========== Ensure Output Directory ==========
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ========== Load Image ==========
image = cv2.imread(IMG_PATH)
if image is None:
    raise FileNotFoundError(f"Image not found at {IMG_PATH}")

image = cv2.resize(image, (128, 128))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_norm = image_rgb / 255.0  # Normalize to [0, 1]

img_tensor = torch.from_numpy(image_norm).permute(2, 0, 1).unsqueeze(0).float()  # [1, 3, 128, 128]

print(f"✅ Loaded Image Tensor Shape: {img_tensor.shape}")

# ========== Initialize Tokenizer ==========
tokenizer = RadialTokenizer()

# ========== Run Tokenization ==========
tokens_192d = tokenizer(img_tensor)  # Shape: [1, 4, 192]
print(f"✅ Tokens Shape: {tokens_192d.shape}")

# ========== Save Token Outputs ==========
torch.save(tokens_192d.cpu(), os.path.join(OUTPUT_DIR, f"{IMG_NAME}_192D.pt"))
print(f"💾 Saved tokens to: {os.path.join(OUTPUT_DIR, f'{IMG_NAME}_192D.pt')}")

# ========== Inspect Tokens ==========
tokens_np = tokens_192d.squeeze(0).detach().cpu().numpy()  # Shape: [4, 192]

# Plot Embeddings Per Ring
plt.figure(figsize=(10, 5))
for i in range(4):
    plt.plot(tokens_np[i], label=f"Ring {i+1}")
plt.title("192D Projected Tokens Per Ring")
plt.xlabel("Embedding Dimension")
plt.ylabel("Value")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# ========== (Optional) Flatten and Export as CSV ==========
flattened = tokens_np.flatten()  # Shape: [4*192]
np.savetxt(os.path.join(OUTPUT_DIR, "patch_features.csv"), flattened[None], delimiter=",", fmt="%.6f")
print(f"💾 Saved flattened features to: {os.path.join(OUTPUT_DIR, 'patch_features.csv')}")